# Interactive Plots 

## Active Interactions: Checkboxes and Sliders

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
from bokeh.io import *
from bokeh.plotting import *

from bokeh.models import *
from bokeh.models.widgets import *

from bokeh.layouts import *
from bokeh.palettes import *

from bokeh.application.handlers import *
from bokeh.application import *
from bokeh.palettes import Inferno256
from bokeh.transform import transform

output_notebook()

Loading BokehJS ...

# Reading in Data

In [3]:
data = pd.read_csv('project_data.csv', parse_dates=['DATE'])
print(data.head(), data.info())
available_people = ['merissa_steps', 'sheridan_steps', 'tova_steps']
data = data.melt(id_vars=['DATE', 'temp'], value_vars=available_people, var_name='person', value_name='steps')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 8 columns):
DATE              122 non-null datetime64[ns]
merissa_dist      122 non-null float64
merissa_steps     122 non-null float64
sheridan_dist     122 non-null float64
sheridan_steps    122 non-null float64
tova_dist         122 non-null float64
tova_steps        122 non-null float64
temp              122 non-null int64
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 7.8 KB
        DATE  merissa_dist  merissa_steps  sheridan_dist  sheridan_steps  \
0 2019-06-01      4.560388    10700.15590       3.674949     8663.697420   
1 2019-06-02      3.315483     7692.00000       2.793206     6468.595866   
2 2019-06-03      5.412403    11560.00000       0.846568     1620.272716   
3 2019-06-04      3.721314     7334.00000       0.747559     1737.605581   
4 2019-06-05      4.872565    10099.04567       1.117429     2359.269982   

   tova_dist  tova_steps  temp  
0   0.008152        

In [4]:
colors = []
for color in ['DarkCyan', 'HotPink', 'Indigo']:
    for entry in range(122):
        colors.append(color)
data['color_by_person'] = colors
data['week'] = data.DATE.dt.week

In [5]:
data.head()

,DATE,temp,person,steps,color_by_person,week
0,2019-06-01,80,merissa_steps,10700.15590,DarkCyan,22
1,2019-06-02,81,merissa_steps,7692.00000,DarkCyan,22
2,2019-06-03,70,merissa_steps,11560.00000,DarkCyan,23
3,2019-06-04,70,merissa_steps,7334.00000,DarkCyan,23
4,2019-06-05,81,merissa_steps,10099.04567,DarkCyan,23


## Plot with Person Select Control

In [6]:
available_people = ['merissa_steps', 'sheridan_steps', 'tova_steps']

def modify_doc(doc):
   
    def make_dataset(person_list):
        
        list_of_frames = []
        for person, frame in zip(person_list, 'abc'[:len(person_list)]):
            frame = data[data['person'] == person]
            list_of_frames.append(frame)
        by_person = pd.concat(list_of_frames)
        
        column_for_week = pd.DataFrame()
        for week_name, data_name, num in zip(['week_' + str(week) for week in range(23,39)],
                                             ['week_data_' + str(week) for week in range(23,39)],
                                             list(range(23,39))):
            column_for_week[week_name] = by_person[by_person['week'] == num]['DATE'].values
            column_for_week[data_name] = by_person[by_person['week'] == num]['steps'].values
            
        column_for_week['color'] = by_person[by_person.week == 23]['color_by_person'].values
        return ColumnDataSource(by_person), ColumnDataSource(column_for_week)

    def tovas_plot(src):
        color_mapper = LinearColorMapper(palette='Inferno256',
                                 low=data.temp.min(), high=data.temp.max())
        color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,-10), title='Temp', height=150)
        p1 = figure(x_axis_type="datetime", title="Everyone's Steps", plot_height=240, plot_width=480,
                    tools=['box_select', 'reset', 'xwheel_zoom', 'xpan'])
        p1.scatter(source=src, x='DATE', y='steps', color=transform('temp', color_mapper))
        p1.add_layout(color_bar, 'right')
        return p1

    def sheridans_plot(src):
        p2 = figure(title="Steps by temperature", plot_height=240, plot_width=480,
                    tools=['box_select', 'reset'])
        p2.scatter(source=src, x='temp', y='steps', line_color='color_by_person', fill_color='color_by_person')
        return p2    

    def merissas_plot(src): 
        weekly_plots = []
        for week_name, data_name, num in zip(['week_' + str(week) for week in range(23,39)],
                                             ['week_data_' + str(week) for week in range(23,39)],
                                             list(range(23,39))):
            p3 = figure(plot_width=240, plot_height=180,
                        x_axis_type='datetime',
                        x_axis_label='Day', y_axis_label='Steps',
                        title='Week {}'.format(num),
                        toolbar_location='below', tools=['hover'], 
                        tooltips=[("Steps", "$y{int}")])
            p3.circle(source=src, x=week_name, y=data_name, color='color', size=10, alpha=.5)
            weekly_plots.append(p3)
        gp = gridplot(weekly_plots, ncols=4)
        return gp
    

    
    def update(attr, old, new):
        people_to_plot = [person_selection.labels[i] for i in 
                            person_selection.active]
        new_src, new_src2 = make_dataset(people_to_plot)

        src.data.update(new_src.data)
        src2.data.update(new_src2.data)
    
    
    
    person_selection = CheckboxGroup(labels=available_people, active = [0, 1, 2])
    person_selection.on_change('active', update)
    
    controls = WidgetBox(person_selection)
    
    initial_people = [person_selection.labels[i] for i in person_selection.active]
    
    src, src2 = make_dataset(initial_people)
    
    p1 = tovas_plot(src)
    p2 = sheridans_plot(src)
    gp = merissas_plot(src2)

    layout = column(controls, row(p1, p2), gp)
    doc.add_root(layout)
    

handler = FunctionHandler(modify_doc)
app = Application(handler)

In [7]:
show(app)